In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import the CRUD module (Ensure your module name and class name match)
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Update with your username and password
username = "aacuser"
password = "Buttercup11"

# Correct initialization with all required parameters
db = AnimalShelter(username, password, "nv-desktop-services.apporto.com", 32558, "AAC", "animals")

# Fetch all records to populate the initial data table
df = pd.DataFrame.from_records(db.read({}))

# MongoDB returns the '_id' column, which we should remove
df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Encode the Grazioso Salvare logo
image_filename = 'Grazioso Salvare Logo.png'  # Use the filename directly
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Dashboard layout
app.layout = html.Div([
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width': '200px'})),
    html.Center(html.B(html.H1('Grazioso Salvare Dashboard'))),
    html.Center(html.H4('By: John Vowcicefski')),
    html.Hr(),
    
    # Filter options
    html.Div([
        html.Label("Select Rescue Type"),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset',
            labelStyle={'display': 'inline-block', 'margin': '10px'}
        )
    ], style={'textAlign': 'center'}),
    
    html.Hr(),
    
    # Data table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_cell={
            'minWidth': '150px', 'width': '150px', 'maxWidth': '150px',
            'whiteSpace': 'normal'
        },
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
        },
        style_data_conditional=[{
            'if': {'column_id': 'age_upon_outcome_in_weeks'},
            'backgroundColor': 'tomato',
            'color': 'white',
        }],
    ),
    
    html.Br(),
    html.Hr(),
    
    # Charts layout
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6'),
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Update the data table based on filter
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if filter_type == 'Reset':
        data = df.to_dict('records')
    else:
        query = {}
        if filter_type == 'Water Rescue':
            query = {'$and': [{'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']}},
                              {'sex_upon_outcome': 'Intact Female'},
                              {'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}}]}
        elif filter_type == 'Mountain or Wilderness Rescue':
            query = {'$and': [{'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']}},
                              {'sex_upon_outcome': 'Intact Male'},
                              {'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}}]}
        elif filter_type == 'Disaster or Individual Tracking':
            query = {'$and': [{'breed': {'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']}},
                              {'sex_upon_outcome': 'Intact Male'},
                              {'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}}]}
        
        filtered_df = pd.DataFrame.from_records(db.read(query))
        filtered_df.drop(columns=['_id'], inplace=True)
        data = filtered_df.to_dict('records')
    
    return data

# Update charts based on filtered data
@app.callback(
    Output('graph-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data')]
)
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    if dff.empty:
        return [dcc.Graph(figure={})]
    
    fig = px.pie(dff, names='breed', title='Preferred Breeds Distribution')
    return [dcc.Graph(figure=fig)]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):  
    if viewData is None or len(viewData) == 0:
        return [dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10)]
    
    dff = pd.DataFrame.from_dict(viewData)
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]
        
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row,13], dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]

# Run the app
app.run_server(mode='inline')